# explore_exercises.ipynb

This is my suggested path forward. You may choose your own adventure for your explore_exercises.ipynb
- If you want a more guided exploration setup, solve the exercises below with seaborn.
- If you want to get more practice with those explore.py functions, use them w/ the Telco dataset

Exercises
Continue in your explore_exercises.ipynb notebook. Use the iris dataset. As always, add, commit, and push your changes.

1. Split your data into train, validate, and test samples.

2. Create a swarmplot using a melted dataframe of all your numeric variables. The x-axis should be the variable name, the y-axis the measure. Add another dimension using color to represent species. Document takeaways from this visualization.

3. Create 4 subplots (2 rows x 2 columns) of scatterplots.

sepal_length x sepal_width
petal_length x petal_width
sepal_area x petal_area
sepal_length x petal_length

4. What are your takeaways? Write them down :)

5. Create a heatmap of each variable layering correlation coefficient on top.

6. Create a scatter matrix visualizing the interaction of each variable.

7. Is the sepal length significantly different in virginica compared to versicolor? Run a statistical experiment to test this.

8. Make sure to include a null hypothesis, alternative hypothesis, results, and summary.

9. What is your takeaway from this statistical testing?

10. Create any other visualizations and run any other statistical tests you think will be helpful in exploring this data set.
